In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%pip install xgboost

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_csv("dataset/train.csv")
testing_data = pd.read_csv("dataset/test.csv")
if 'Survived' not in testing_data.columns:
    testing_data['Survived'] = 0

In [4]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
def Preprocess(df_test, df_train=pd.read_csv("dataset/train.csv")):    
    df = pd.concat([df_train, df_test], axis=0)
    df = df.drop('Name', axis = 1)
    df = df.drop('Ticket', axis = 1)
    df['Cabin'] = df['Cabin'].fillna('X000')
    df['Embarked'] = df['Embarked'].fillna('X')
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())

    # Apply the split_cabin_multiple function to each row
    df['cabin_letter'] = df['Cabin'].str.extract(r'([a-zA-Z]+)', expand=False)
    df['cabin_number'] = df['Cabin'].str.extract(r'(\d+)', expand=False)

    # One-hot encode the "cabin_letter" column
    df_encoded = pd.get_dummies(df, columns=['cabin_letter'], prefix='cabin')
    df_encoded = pd.get_dummies(df_encoded, columns=['Embarked'], prefix='Embarked')
    df_encoded = pd.get_dummies(df_encoded, columns=['Sex'])
    # Drop the original "cabin" column
    df_encoded = df_encoded.drop(columns=['Cabin'])
    df_encoded = df_encoded.drop(columns=['cabin_X'])
    df_encoded = df_encoded.drop(columns=['Embarked_X'])
    df = df_encoded
    df['cabin_number'] = df['cabin_number'].fillna(0)
    df['cabin_number'] = pd.to_numeric(df['cabin_number'])

    #Feature Engineering
    df['Pclass_bin_Fare'] = df['Fare'] // df['Pclass'] 
    df['Pclass_bin_Sex'] =  df['Pclass'] - df['Sex_female']

    df_train = df[:len(df_train)]
    df_test = df[len(df_train):]

    df_test = df_test.drop(columns=['Survived'])

    return df_train, df_test

In [7]:
df, testing_data = Preprocess(testing_data)


In [8]:
df.isna().sum()

PassengerId        0
Survived           0
Pclass             0
Age                0
SibSp              0
Parch              0
Fare               0
cabin_number       0
cabin_A            0
cabin_B            0
cabin_C            0
cabin_D            0
cabin_E            0
cabin_F            0
cabin_G            0
cabin_T            0
Embarked_C         0
Embarked_Q         0
Embarked_S         0
Sex_female         0
Sex_male           0
Pclass_bin_Fare    0
Pclass_bin_Sex     0
dtype: int64

In [9]:
df.corr()['Survived']

PassengerId       -0.005007
Survived           1.000000
Pclass            -0.338481
Age               -0.070323
SibSp             -0.035322
Parch              0.081629
Fare               0.257307
cabin_number       0.229756
cabin_A            0.022287
cabin_B            0.175095
cabin_C            0.114652
cabin_D            0.150716
cabin_E            0.145321
cabin_F            0.057935
cabin_G            0.016040
cabin_T           -0.026456
Embarked_C         0.168240
Embarked_Q         0.003650
Embarked_S        -0.155660
Sex_female         0.543351
Sex_male          -0.543351
Pclass_bin_Fare    0.267823
Pclass_bin_Sex    -0.533994
Name: Survived, dtype: float64

In [10]:
X = df.drop(columns=['Survived'])
y = df['Survived']
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, y, test_size=0.2)
Train_Y = np.reshape(Train_Y,(-1, 1))

In [14]:
model = LogisticRegression()
model.fit(Train_X, Train_Y)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
model_2 = XGBClassifier(enable_categorical = True)
model_2.fit(Train_X, Train_Y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [17]:
model_3 = DecisionTreeClassifier()
model_3.fit(Train_X, Train_Y)


DecisionTreeClassifier()

In [18]:
model_4 = RandomForestClassifier()
model_4.fit(Train_X, Train_Y)


/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [19]:
accuracy_score(Test_Y, model.predict(Test_X))

0.7374301675977654

In [20]:
accuracy_score(Test_Y, model_2.predict(Test_X))

0.770949720670391

In [21]:
accuracy_score(Test_Y, model_3.predict(Test_X))

0.7486033519553073

In [22]:
accuracy_score(Test_Y, model_4.predict(Test_X))

0.776536312849162

In [23]:
pred = model_4.predict(testing_data)

final = pd.DataFrame()
final['PassengerId'] = testing_data['PassengerId']
final['Survived'] = pred

# Write DataFrame to a CSV file without index
final.to_csv('dataset/output.csv', index=False)